In [ ]:
import os.path
import openpyxl

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.


SAMPLE_SPREADSHEET_ID = '10z91uk1LeNnFIHiV9UggGw8nGKKLDj5bqqr5zzbNpOQ'
SAMPLE_RANGE_NAME = "PG1!B:J"


"""
1 - Antes de começar crie uma pasta chamada 'security' e adicione o arquivo de credenciais do Google Cloud.
2 - Renomeie o arquivo para credentials.json ou altere o nome do arquivo no código.
"""
def main():
  creds = None
  if os.path.exists("security\\token.json"):
    creds = Credentials.from_authorized_user_file("security\\token.json", SCOPES)
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "security\\credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    with open("security\\token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    
    val = result['values']
    
    book = openpyxl.Workbook()
    page = book['Sheet']
    if not os.path.exists('archives'):
      os.mkdir('archives')
      
    for linha in val:
        print(linha)
        page.append(linha)
    
    book.save('archives\\vals.xlsx')
      
  except HttpError as err:
    print(err)


if __name__ == "__main__":
  main()

In [ ]:
import pandas as pd 
df = pd.read_excel('archives\\vals.xlsx')


display(df)

In [ ]:
import plotly.express as px
meses = df.drop(columns=['dias','total'])

for mes in meses:
  print(mes)
  grafico = px.histogram(meses,x=mes, color=mes)
  grafico.show()